<a href="https://colab.research.google.com/github/Aljesh-basnet/gif_auto_audio_description/blob/Dev/Aljesh_3DCNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import imageio
import numpy as np

In [123]:

import pandas as pd
from google.colab import drive

# Step 1: Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Step 2: Load TSV file into a DataFrame without using the first line as the header
file_path = '/content/drive/My Drive/tgif-v1.0.tsv'  # Correct file path
df = pd.read_csv(file_path, sep='\t', header=None)  # Load TSV

# Display the first few rows of the DataFrame
df.head()


Mounted at /content/drive


,0,1
0,https://38.media.tumblr.com/9f6c25cc350f12aa74...,"a man is glaring, and someone with sunglasses ..."
1,https://38.media.tumblr.com/9ead028ef62004ef6a...,a cat tries to catch a mouse on a tablet
2,https://38.media.tumblr.com/9f43dc410be85b1159...,a man dressed in red is dancing.
3,https://38.media.tumblr.com/9f659499c8754e40cf...,an animal comes close to another in the jungle
4,https://38.media.tumblr.com/9ed1c99afa7d714118...,a man in a hat adjusts his tie and makes a wei...


In [124]:

df.columns = ['Image', 'Text']
df.head()

,Image,Text
0,https://38.media.tumblr.com/9f6c25cc350f12aa74...,"a man is glaring, and someone with sunglasses ..."
1,https://38.media.tumblr.com/9ead028ef62004ef6a...,a cat tries to catch a mouse on a tablet
2,https://38.media.tumblr.com/9f43dc410be85b1159...,a man dressed in red is dancing.
3,https://38.media.tumblr.com/9f659499c8754e40cf...,an animal comes close to another in the jungle
4,https://38.media.tumblr.com/9ed1c99afa7d714118...,a man in a hat adjusts his tie and makes a wei...


In [125]:

# Function to extract GIF and save frames as image links
def extract_and_save_frames(gif_path, num_frames=10):
    try:
        frames = extract_frames(gif_path, num_frames)
        image_links = []
        for i, frame in enumerate(frames):
            image_filename = f"{os.path.basename(gif_path)}_frame_{i}.png"
            imageio.imwrite(image_filename, frame)  # Save the frame as a PNG file
            image_links.append(image_filename)  # Store the image filename (or URL)
        return image_links
    except Exception as e:
        print(f"Error extracting and saving frames from {gif_path}: {e}")
        return []



In [126]:
# Create a new column in the DataFrame to store image links
n_rows=25
new_df= df.iloc[:n_rows].copy()
new_df['Image_Links'] = new_df['Image'].apply(extract_and_save_frames)


In [127]:
new_df.head()

,Image,Text,Image_Links
0,https://38.media.tumblr.com/9f6c25cc350f12aa74...,"a man is glaring, and someone with sunglasses ...",[tumblr_mevmyaKtDf1rgvhr8o1_500.gif_frame_0.pn...
1,https://38.media.tumblr.com/9ead028ef62004ef6a...,a cat tries to catch a mouse on a tablet,[tumblr_nok4eeONTv1s2yegdo1_400.gif_frame_0.pn...
2,https://38.media.tumblr.com/9f43dc410be85b1159...,a man dressed in red is dancing.,[tumblr_mllh01J96X1s9npefo1_250.gif_frame_0.pn...
3,https://38.media.tumblr.com/9f659499c8754e40cf...,an animal comes close to another in the jungle,[tumblr_nqlr0rn8ox1r2r0koo1_400.gif_frame_0.pn...
4,https://38.media.tumblr.com/9ed1c99afa7d714118...,a man in a hat adjusts his tie and makes a wei...,[tumblr_mvtuwlhSkE1qbnleeo1_500.gif_frame_0.pn...


In [128]:
# # prompt: convert frames to numbers for cnn lstm model

# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, TimeDistributed
# from tensorflow.keras.models import Model
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.utils import to_categorical
# import imageio
# import numpy as np
# from google.colab import drive
# import os

# # Assuming you have new_df with 'Image_Links' column

# # Load pre-trained ResNet50 model (without the top classification layer)
# resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# def convert_frames_to_numbers(image_links):
#     """
#     Converts a list of image links to a list of numerical feature vectors using ResNet50.

#     Args:
#       image_links: A list of image file paths.

#     Returns:
#       A list of numerical feature vectors.
#     """
#     feature_vectors = []
#     for image_link in image_links:
#         try:
#             img = tf.keras.utils.load_img(image_link, target_size=(224, 224))
#             img_array = tf.keras.utils.img_to_array(img)
#             img_array = np.expand_dims(img_array, axis=0)
#             img_array = tf.keras.applications.resnet50.preprocess_input(img_array)
#             features = resnet_model.predict(img_array)
#             feature_vectors.append(features.flatten())
#         except Exception as e:
#             print(f"Error processing image {image_link}: {e}")
#             feature_vectors.append(np.zeros(2048))  # Replace with a default vector if an error occurs
#     return feature_vectors

# # Create a new column to store the numerical features for each list of image links
# new_df['Image_Features'] = new_df['Image_Links'].apply(convert_frames_to_numbers)

# # Now, you have numerical features for your frames.
# # You can use these features as input to your CNN-LSTM model.

# new_df.head()


In [129]:
# # prompt: fit new_df to 3dcnn model

# import numpy as np
# from tensorflow.keras.preprocessing.image import img_to_array, load_img

# def preprocess_gif_frames(image_links, target_size=(128, 128)):
#     frames = []
#     for link in image_links:
#         try:
#             img = load_img(link, target_size=target_size)
#             img_array = img_to_array(img)
#             frames.append(img_array)
#         except Exception as e:
#             print(f"Error processing image {link}: {e}")
#     return np.array(frames)


In [130]:
# # # Separate X and y
# X = new_df['Image_Features'] # Image links as input
# y = new_df['Text']  # Text as output

# X.head()


In [131]:
X = new_df['Image_Links']
y = new_df['Text']

In [132]:
X.shape

(25,)

In [133]:
y.shape

(25,)

In [134]:
X.head()

,Image_Links
0,[tumblr_mevmyaKtDf1rgvhr8o1_500.gif_frame_0.pn...
1,[tumblr_nok4eeONTv1s2yegdo1_400.gif_frame_0.pn...
2,[tumblr_mllh01J96X1s9npefo1_250.gif_frame_0.pn...
3,[tumblr_nqlr0rn8ox1r2r0koo1_400.gif_frame_0.pn...
4,[tumblr_mvtuwlhSkE1qbnleeo1_500.gif_frame_0.pn...


In [135]:
y.head()

,Text
0,"a man is glaring, and someone with sunglasses ..."
1,a cat tries to catch a mouse on a tablet
2,a man dressed in red is dancing.
3,an animal comes close to another in the jungle
4,a man in a hat adjusts his tie and makes a wei...


In [150]:
# prompt: video to array for x

import numpy as np
import cv2

def video_to_array(image_links, frame_size=(128, 128), max_frames=5):
    """
    Converts a list of image links into a NumPy array of frames, suitable for 3D CNN input.

    Args:
    - image_links (list): A list of image file paths.
    - frame_size (tuple): Desired size of each frame (width, height).
    - max_frames (int): Maximum number of frames to extract from the video.

    Returns:
    - video_array (np.ndarray): A 4D NumPy array of shape (num_frames, height, width, channels).
    """
    frames = []
    for link in image_links:
        try:
            frame = cv2.imread(link)
            if frame is not None:
                # Resize the frame to the desired size (frame_size[1], frame_size[0])
                frame_resized = cv2.resize(frame, frame_size)

                # Convert BGR (OpenCV default) to RGB
                frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)

                # Append the frame to the list
                frames.append(frame_rgb)

            if len(frames) >= max_frames:
                break

        except Exception as e:
            print(f"Error processing image {link}: {e}")

    # Pad frames if there are fewer than max_frames
    while len(frames) < max_frames:
        frames.append(np.zeros((frame_size[1], frame_size[0], 3), dtype=np.uint8))


    # Convert the list of frames to a NumPy array
    video_array = np.array(frames)

    # Normalize pixel values to range [0, 1]
    video_array = video_array.astype('float32') / 255.0

    return video_array

# Example usage:
# Assuming 'X' is a column with image links in your DataFrame

X_array = X.apply(video_to_array)


print(X_array.shape)
X_array = np.array([arr for arr in X_array])

print(X_array.shape)


(25,)
(25, 5, 128, 128, 3)


In [151]:
X_array

array([[[[[0.58431375, 0.5568628 , 0.5058824 ],
          [0.5921569 , 0.5686275 , 0.5176471 ],
          [0.5921569 , 0.5647059 , 0.5137255 ],
          ...,
          [0.44313726, 0.42745098, 0.4       ],
          [0.6745098 , 0.65882355, 0.627451  ],
          [0.8666667 , 0.8509804 , 0.8235294 ]],

         [[0.5921569 , 0.5647059 , 0.5137255 ],
          [0.59607846, 0.5686275 , 0.5176471 ],
          [0.5882353 , 0.56078434, 0.5137255 ],
          ...,
          [0.52156866, 0.5058824 , 0.48235294],
          [0.5647059 , 0.54901963, 0.5176471 ],
          [0.5764706 , 0.56078434, 0.52156866]],

         [[0.58431375, 0.56078434, 0.5137255 ],
          [0.57254905, 0.54509807, 0.5019608 ],
          [0.5764706 , 0.54901963, 0.49411765],
          ...,
          [0.34117648, 0.3254902 , 0.30588236],
          [0.4392157 , 0.41960785, 0.4       ],
          [0.56078434, 0.54509807, 0.5176471 ]],

         ...,

         [[0.        , 0.        , 0.        ],
          [0.        ,

In [152]:
# prompt: preprocess y

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming 'new_df' contains your DataFrame with text captions
y_captions = new_df['Text'].tolist()

# Tokenize the captions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(y_captions)
y_sequences = tokenizer.texts_to_sequences(y_captions)

# Pad sequences to a fixed length
max_caption_length = 20  # You might need to adjust this based on your data
y_data = pad_sequences(y_sequences, maxlen=max_caption_length, padding='post')

# Convert to categorical if you're using a categorical cross-entropy loss
# num_words = len(tokenizer.word_index) + 1
# y_data = to_categorical(y_data, num_classes=num_words)

print("Shape of y_data:", y_data.shape)


Shape of y_data: (25, 20)


In [153]:
X_array

array([[[[[0.58431375, 0.5568628 , 0.5058824 ],
          [0.5921569 , 0.5686275 , 0.5176471 ],
          [0.5921569 , 0.5647059 , 0.5137255 ],
          ...,
          [0.44313726, 0.42745098, 0.4       ],
          [0.6745098 , 0.65882355, 0.627451  ],
          [0.8666667 , 0.8509804 , 0.8235294 ]],

         [[0.5921569 , 0.5647059 , 0.5137255 ],
          [0.59607846, 0.5686275 , 0.5176471 ],
          [0.5882353 , 0.56078434, 0.5137255 ],
          ...,
          [0.52156866, 0.5058824 , 0.48235294],
          [0.5647059 , 0.54901963, 0.5176471 ],
          [0.5764706 , 0.56078434, 0.52156866]],

         [[0.58431375, 0.56078434, 0.5137255 ],
          [0.57254905, 0.54509807, 0.5019608 ],
          [0.5764706 , 0.54901963, 0.49411765],
          ...,
          [0.34117648, 0.3254902 , 0.30588236],
          [0.4392157 , 0.41960785, 0.4       ],
          [0.56078434, 0.54509807, 0.5176471 ]],

         ...,

         [[0.        , 0.        , 0.        ],
          [0.        ,

In [177]:
num_words = len(tokenizer.word_index) + 1  # Calculate the number of unique words
y_data_categorical = to_categorical(y_data, num_classes=num_words) # Use num_words as num_classes
y_data_categorical.shape

(25, 20, 133)

In [179]:
# Example usage
input_shape = (5, 128, 128, 3)  # (num_frames, height, width, channels)
num_classes = num_words  # Number of classes for classification
num_classes

133

In [186]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, LSTM, TimeDistributed, Reshape


def create_3dcnnlstm(input_shape, num_classes):
    """
    Creates a 3D CNN-LSTM model for spatiotemporal tasks (e.g., video classification).

    Parameters:
    - input_shape: Tuple of the shape (num_frames, height, width, channels), e.g., (16, 128, 128, 3)
    - num_classes: The number of output classes for classification.

    Returns:
    - model: A compiled Keras model.
    """

    model = Sequential()

    # 3D CNN layers for spatial and temporal feature extraction
    model.add(Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu', input_shape=input_shape, padding='same'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same'))  # Pool only over height and width, not depth

    model.add(Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu', padding='same'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same'))

    model.add(Conv3D(filters=128, kernel_size=(3, 3, 3), activation='relu', padding='same'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same'))

    # Flatten the 3D CNN output and reshape for the LSTM
    model.add(Flatten())
    model.add(Reshape((20, -1))) # Reshape to (timesteps, features) for LSTM

    # LSTM layers for temporal modeling
    model.add(LSTM(128, return_sequences=True))  # You can set return_sequences=True if stacking LSTMs

    # Dropout for regularization
    model.add(Dropout(0.5))

    # Fully connected layer
    model.add(TimeDistributed(Dense(128, activation='relu')))

    # Output layer with softmax for multi-class classification
    model.add(TimeDistributed(Dense(num_classes, activation='softmax')))

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model



model = create_3dcnnlstm(input_shape, num_classes)
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d_64 (Conv3D)                   │ (None, 5, 128, 128, 32)     │           2,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_62 (MaxPooling3D)      │ (None, 5, 64, 64, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_65 (Conv3D)                   │ (None, 5, 64, 64, 64)       │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_63 (MaxPooling3D)      │ (None, 5, 32, 32, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_66 (Conv3D)                   │ (None, 5, 32, 32, 128)      │         221,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_64 (MaxPooling3D)      │ (None, 5, 16, 16, 128)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_18 (Flatten)                 │ (None, 163840)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_18 (Reshape)                 │ (None, 20, 8192)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_18 (LSTM)                       │ (None, 20, 128)             │       4,260,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 20, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_2 (TimeDistributed) │ (None, 20, 128)             │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_3 (TimeDistributed) │ (None, 20, 133)             │          17,157 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,573,317 (17.45 MB)

 Trainable params: 4,573,317 (17.45 MB)

 Non-trainable params: 0 (0.00 B)

In [187]:
# Fit the model
model.fit(X_array, y_data_categorical, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step - accuracy: 0.0125 - loss: 4.8986 - val_accuracy: 0.5100 - val_loss: 4.4346
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 18s 18s/step - accuracy: 0.4600 - loss: 4.4159 - val_accuracy: 0.5100 - val_loss: 3.6736
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 21s 21s/step - accuracy: 0.4725 - loss: 3.7427 - val_accuracy: 0.5100 - val_loss: 3.2077
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 24s 24s/step - accuracy: 0.4725 - loss: 3.3839 - val_accuracy: 0.5100 - val_loss: 3.1426
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 18s 18s/step - accuracy: 0.4725 - loss: 3.3495 - val_accuracy: 0.5100 - val_loss: 3.0765
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 23s 23s/step - accuracy: 0.4725 - loss: 3.2912 - val_accuracy: 0.5100 - val_loss: 2.9523
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 38s 38s/step - accuracy: 0.4725 - loss: 3.1418 - val_accuracy: 0.5100 - val_loss: 2.9314
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step - accuracy: 0.4725 - loss: 3.1275 - val_accuracy: 0.5100 - val_loss: 2.9205


In [111]:
y_data.shape

(25, 20)

In [190]:
# prompt: calculate rouge score, blue score

import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import imageio
import numpy as np
from google.colab import drive
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, LSTM, TimeDistributed, Reshape
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer

# ... (Your existing code for data preprocessing and model training) ...


# Assuming you have 'y_data' (padded sequences of predicted captions) and 'y_captions' (actual captions)
# Convert predicted sequences back to text using the tokenizer
y_pred_texts = []
for seq in y_data:
    text = tokenizer.sequences_to_texts([seq])[0]  # Convert sequence back to text
    y_pred_texts.append(text)

# Calculate BLEU score
bleu_score = corpus_bleu([[ref] for ref in y_captions], y_pred_texts)
print(f"BLEU Score: {bleu_score}")

# Calculate ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []
for ref, pred in zip(y_captions, y_pred_texts):
    scores = scorer.score(ref, pred)
    rouge_scores.append(scores)

# Calculate the average ROUGE scores
avg_rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores])
avg_rouge2 = np.mean([score['rouge2'].fmeasure for score in rouge_scores])
avg_rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores])

print(f"Average ROUGE-1 Score: {avg_rouge1}")
print(f"Average ROUGE-2 Score: {avg_rouge2}")
print(f"Average ROUGE-L Score: {avg_rougeL}")


BLEU Score: 0.9835161237051986
Average ROUGE-1 Score: 1.0
Average ROUGE-2 Score: 1.0
Average ROUGE-L Score: 1.0


In [193]:
import pickle as pkl
pkl.dump(model, open('model.pkl', 'wb'))
# Load the saved model
loaded_model = pkl.load(open('model.pkl', 'rb'))



In [203]:
# prompt: use loaded_model to generate captions for test.gif

import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import imageio
import numpy as np
from google.colab import drive
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, LSTM, TimeDistributed, Reshape
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import pickle as pkl
import os


# Assuming you have 'loaded_model' and 'tokenizer' from previous steps

def generate_caption_for_gif(gif_path, model, tokenizer, max_caption_length=20):
    """
    Generates a caption for a GIF using a trained model.

    Args:
      gif_path: The path to the GIF file.
      model: The trained model for caption generation.
      tokenizer: The tokenizer used during training.
      max_caption_length: The maximum length of the generated caption.

    Returns:
      The generated caption.
    """

    # Step 1: Extract frames from the GIF and convert to numpy array
    frames = extract_and_save_frames(gif_path)
    video_array = video_to_array(frames)
    video_array = np.expand_dims(video_array, axis=0)

    # Step 2: Use the model to predict the caption
    predicted_sequence = model.predict(video_array)[0]


    # Step 3: Convert the predicted sequence back to text
    predicted_sequence = np.argmax(predicted_sequence, axis=-1)
    predicted_caption = tokenizer.sequences_to_texts([predicted_sequence])[0]


    return predicted_caption

# Example usage
test_gif_path = '/content/drive/My Drive/test.gif'  # Replace with the actual path to your test GIF
if os.path.exists(test_gif_path):
  predicted_caption = generate_caption_for_gif(test_gif_path, loaded_model, tokenizer)
  print(f"Predicted Caption for test.gif: {predicted_caption}")
else:
  print("test.gif not found")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step
Predicted Caption for test.gif: 
